# Normalization 

The normalization for training and validation dataset. This normalization funtions (class) will be called in the data generator to normalize data in every batch. In this normalization script, we need first to calculate the mean and std of one year dataset. Since the calculation will cost a lot of time. We find two relative efficiently apporaches to deal with normalization process.

1. Subsample from all datasets. This approach is a approximation to the global mean and std, which was implemented in CBRAIN by Stephen Rasp.

2. Divide all datasets into several subsets. Calculate mean of every subsets (sub_mean), then calculate the global mean and std based on several sub_means. This approach is the same as calculated directly with all datasets.



Objective: 
    
    the mean and std of one year datasets.

 Function: 

    1. Divide one year datasets into 12 months
    2. Calculate 12 month mean values, save the mean and release the RAM after calculation of the mean
    3. Average 12 means to get the global mean
    4. Based on the equation to calculate the global std. 
    
    
$S = \sqrt{\frac{\sum (x_{i} - \bar{x})^2}{n-1} } $


## 1. Subsample normalization

We subsampled first three days from every month to calculate the approixmate mean and std, which was implemented in the CBRAIN by Stephen Rasp. In this approach, the normalization process is the same as Jerry's previous workflow. So we don't need to change anything.
> Note that in Jerry's workflow he sampled the data spatially with the arguement `space=8`. We should cancel spatial sampling (set `space=1`) to ensure a sufficient number of samples to calculate mean and std. We don't need to change the `space` at this time since the training data also be sampled. So We just follow Jerry's workflow without change anything. If we want to use all data for training, we should modify the normalization process.

In [3]:
# load library
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from tqdm import tqdm
from preprocessing_functions import *

In [4]:
data_path = '/ocean/projects/atm200007p/jlin96/longSPrun/'
spData_train_input_3d = ! ls {data_path} | grep '0[123]-00000.nc' # first three days of every month
spData_train_input_3d = [data_path + s for s in spData_train_input_3d[:36]]
spData_train_input_3d = xr.open_mfdataset(spData_train_input_3d)

In [5]:
# subsample data with `space=8`
spData_train_input_3d = combine_arrays(make_nn_input(spData_train_input_3d, family = "relative", print_diagnostics = True))

loaded in data
starting for loop


100%|██████████| 64/64 [06:55<00:00,  6.49s/it]


nntbp
(30, 1727, 64, 128)
nnqbp
(30, 1727, 64, 128)
lhflx
(1, 1727, 64, 128)
shflx
(1, 1727, 64, 128)
ps
(1, 1727, 64, 128)
solin
(1, 1727, 64, 128)
newhum
(30, 1727, 64, 128)
oldhum
(30, 1727, 64, 128)
nnInput
(64, 1727, 64, 16)
Mean relative humidity conversion error: 0.004416230272216098
Variance for relative humidity conversion error: 0.000562019650883235
nntbp.shape: (30, 1727, 64, 128)
nnqbp.shape: (30, 1727, 64, 128)
lhflx.shape: (1, 1727, 64, 128)
shflx.shape: (1, 1727, 64, 128)
ps.shape: (1, 1727, 64, 128)
solin.shape: (1, 1727, 64, 128)
newhum.shape: (30, 1727, 64, 128)
oldhum.shape: (30, 1727, 64, 128)
nnInput.shape: (64, 1727, 64, 16)



In [8]:
# reshape
spData_train_input_3d = reshape_input(spData_train_input_3d)
print(spData_train_input_3d.shape)

(64, 1767424)


In [11]:
X_train_3d, inp_sub_3d, inp_div_3d = normalize_input_train(X_train = spData_train_input_3d, save_files = False)

X_train shape: 
(1767424, 64)
INP_SUB shape: 
(64, 1)
INP_DIV shape: 
(64, 1)


In [13]:
del X_train, X_train_3d

## Normalization with all dataset
In order to estimate the errors between sampling normalization and global normalization, we need to calculate the mean and std using all data. 

In [18]:
spData_train_input = ! ls {data_path} | grep 'h1.0000-*'
spData_train_input = [data_path + s for s in spData_train_input]
spData_train_input = xr.open_mfdataset(spData_train_input)

AttributeError: 'Dataset' object has no attribute 'shape'

In [20]:
# reshape
spData_train_input = combine_arrays(make_nn_input(spData_train_input, family = "relative", print_diagnostics = True))
spData_train_input = reshape_input(spData_train_input)
print(spData_train_input.shape)

loaded in data
starting for loop


100%|██████████| 64/64 [1:07:18<00:00, 63.10s/it]


nntbp
(30, 17519, 64, 128)
nnqbp
(30, 17519, 64, 128)
lhflx
(1, 17519, 64, 128)
shflx
(1, 17519, 64, 128)
ps
(1, 17519, 64, 128)
solin
(1, 17519, 64, 128)
newhum
(30, 17519, 64, 128)
oldhum
(30, 17519, 64, 128)
nnInput
(64, 17519, 64, 16)
Mean relative humidity conversion error: 0.004461620695778564
Variance for relative humidity conversion error: 0.0005681880569581219
nntbp.shape: (30, 17519, 64, 128)
nnqbp.shape: (30, 17519, 64, 128)
lhflx.shape: (1, 17519, 64, 128)
shflx.shape: (1, 17519, 64, 128)
ps.shape: (1, 17519, 64, 128)
solin.shape: (1, 17519, 64, 128)
newhum.shape: (30, 17519, 64, 128)
oldhum.shape: (30, 17519, 64, 128)
nnInput.shape: (64, 17519, 64, 16)

(64, 17938432)


In [23]:
spData_train_input_3d.shape, spData_train_input.shape

((64, 1767424), (64, 17938432))

In [24]:
X_train, inp_sub, inp_div = normalize_input_train(X_train = spData_train_input, save_files = False)

X_train shape: 
(17938432, 64)
INP_SUB shape: 
(64, 1)
INP_DIV shape: 
(64, 1)


In [42]:
# Difference
diff_sub = (inp_sub - inp_sub_3d)
diff_div = (inp_div - inp_div_3d)

# relative error
diff_sub_re = diff_sub / inp_sub
diff_div_re = diff_div / inp_div

In [44]:
norm_path = '../training/norm_files/'
np.savetxt(norm_path + "inp_sub_3d.txt", inp_sub_3d, delimiter=',')
np.savetxt(norm_path + "inp_div_3d.txt", inp_div_3d, delimiter=',')
np.savetxt(norm_path + "inp_sub.txt", inp_sub, delimiter=',')
np.savetxt(norm_path + "inp_div.txt", inp_div, delimiter=',')
np.savetxt(norm_path + "inp_sub_re.txt", diff_sub_re, delimiter=',')
np.savetxt(norm_path + "inp_div_re.txt", diff_div_re, delimiter=',')

In [40]:
diff_div_ratio

array([[ 2.64231081e-03],
       [ 2.44049322e-03],
       [ 3.79727147e-03],
       [-1.61518298e-03],
       [-3.68291146e-03],
       [-2.28174110e-03],
       [-5.82471073e-04],
       [ 1.65606086e-03],
       [ 2.40968722e-06],
       [-1.67228977e-03],
       [-1.19925376e-03],
       [ 9.73574745e-04],
       [ 5.31810213e-03],
       [ 8.89999689e-03],
       [ 1.18258173e-02],
       [ 1.54700750e-02],
       [ 1.72744807e-02],
       [ 1.73006562e-02],
       [ 1.65300093e-02],
       [ 1.50818047e-02],
       [ 1.30678423e-02],
       [ 1.03079726e-02],
       [ 8.49296786e-03],
       [ 7.49021928e-03],
       [ 6.75449250e-03],
       [ 6.17895239e-03],
       [ 6.00480277e-03],
       [ 6.20741673e-03],
       [ 6.95788353e-03],
       [ 8.48935096e-03],
       [-7.80933463e-03],
       [-8.98542308e-03],
       [ 3.84467087e-03],
       [-7.52525962e-02],
       [-5.91047126e-02],
       [-4.23495474e-02],
       [-2.47092095e-02],
       [-6.69444050e-03],
       [-1.6

## 2. Calculated from month average 
1. Load month files separately, calculate the mean for each month, save mean values and del dataset in RAM 
2. Avarage the month means to get global mean
3. Calculate the std based on equation

In [ ]:
train_mu_m = []
for i in range(1,13):
    spData_train_input_m = 'spData_train_input_' + i + 'm'
    spData_train_input_m = combine_arrays(make_nn_input(load_data(month = i, year = 0, data_path = data_path), family = "relative", print_diagnostics = True))
    spData_train_input_m = reshape_input(spData_train_input_m)
    X_train, inp_sub, inp_div = normalize_input_train(X_train = spData_train_input_m, save_files = False)
    train_mu_m.append(inp_sub)
    train_mu_m = np.array(train_mu_m)
    del X_train
    
train_mu = np.mean(train_mu_m, axis=1)


In [ ]:
def normalize_input_train(X_train, reshaped = True, norm = "standard", save_files = False, norm_path = "../training/norm_files/", save_path = "../training/training_data/"):
    if reshaped:
        train_mu = np.mean(X_train, axis = 1)[:, np.newaxis]
        train_std = np.std(X_train, axis = 1)[:, np.newaxis]
        train_min = X_train.min(axis = 1)[:, np.newaxis]
        train_max = X_train.max(axis = 1)[:, np.newaxis]
    
    else:
        train_mu = np.mean(X_train, axis = (1,2,3))[:, np.newaxis]
        train_std = np.std(X_train, axis = (1,2,3))[:, np.newaxis]
        train_min = X_train.min(axis = (1,2,3))[:, np.newaxis]
        train_max = X_train.max(axis = (1,2,3))[:, np.newaxis]
        
    if norm == "standard":
        inp_sub = train_mu
        inp_div = train_std
        
    elif norm == "range":
        inp_sub = train_min
        inp_div = train_max - train_min
        
    #normalizing
    X_train = ((X_train - inp_sub)/inp_div).transpose()
    #normalized
    
    print("X_train shape: ")
    print(X_train.shape)
    print("INP_SUB shape: ")
    print(inp_sub.shape)
    print("INP_DIV shape: ")
    print(inp_div.shape)
    
    if save_files:
        with open(save_path + "train_input.npy", 'wb') as f:
            np.save(f, np.float32(X_train))
        np.savetxt(norm_path + "inp_sub.txt", inp_sub, delimiter=',')
        np.savetxt(norm_path + "inp_div.txt", inp_div, delimiter=',')
    
    return X_train, inp_sub, inp_div